In [81]:
import pandas as pd

In [82]:
ffddf = pd.read_csv('FFBallers_DST_2022.csv')
fpddf = pd.read_csv('FPros_DST_2022.csv')

In [83]:
#remove unwanted cols
ffddf.drop(columns=['Name', 'Jason', 'Mike'], inplace=True)

In [84]:
fpddf.drop(columns=['PLAYER NAME', 'PTS VS.'], inplace=True)

In [85]:
#capitalize FFB COl Headings to match FPROS
ffddf.columns = [x.upper() for x in ffddf.columns]

In [86]:
#rename FFB columns
ffddf.rename(columns={'RANK': 'FFB RK'},inplace=True)

In [87]:
#rename Fpros columns
fpddf.rename(columns={'RK': 'FPS RK', 'TIERS': 'TIER'},inplace=True)

In [88]:
ffddf.head()

,TEAM,FFB RK,ANDY
0,BUF,1,1
1,TB,2,3
2,LAR,3,2
3,DEN,4,5
4,NO,5,4


In [89]:
jags = fpddf['TEAM'] == 'JAC'
fpddf[jags]

,FPS RK,TIER,TEAM,FAN PTS,SACK,SAFE,INT,FUMR,DEF TD,BLK KICK,RET TD
27,28,5,JAC,64,32,0,7,2,0,0,0


In [90]:
fpddf.loc[jags, 'TEAM'] = 'JAX'

In [91]:
#combine the data frames
combined_ddf = pd.merge(fpddf, ffddf, on='TEAM')

In [92]:
#make an average rank column
combined_ddf['AVG RK'] = (combined_ddf['FPS RK'] + combined_ddf['FFB RK'])/2

In [93]:
#set index to avg rank
#combined_ddf.set_index('AVG RK', inplace=True)

In [94]:
#sort values first by FAN PTS in order to make a new col that ranks the team's finishing spot last year 
combined_ddf.sort_values(by=['FAN PTS'], ascending=False, inplace=True)
#combined_ddf.sort_values(by=['AVG RK', 'ANDY'], inplace=True)

In [95]:
combined_ddf.head()

,FPS RK,TIER,TEAM,FAN PTS,SACK,SAFE,INT,FUMR,DEF TD,BLK KICK,RET TD,FFB RK,ANDY,AVG RK
8,9,3,DAL,185,41,0,26,8,6,0,0,10,9,9.5
5,6,2,NE,161,36,0,23,7,3,0,0,21,20,13.5
9,10,3,MIA,155,48,0,14,13,5,0,0,24,26,17.0
0,1,1,BUF,150,42,0,19,11,1,0,0,1,1,1.0
6,7,2,NO,145,46,0,18,7,2,0,0,5,4,6.0


In [96]:
#function to rank where teams finished last year
#set the counter to 0
dst_rank = 0

In [97]:
def last_season_dst_rank(team):
    
    #counter set above
    #just adds one to the rank as it goes down the rows filtered by points scored last year
    global dst_rank
    dst_rank +=1
    return 'DST' + ' ' + str(dst_rank)
    

In [98]:
#run fxn on pos col and generate new column
combined_ddf['FINISH'] = combined_ddf['TEAM'].apply(last_season_dst_rank)

In [99]:
#set sort to avg rank and secondarily by last year's points 
combined_ddf.sort_values(by=['AVG RK', 'FAN PTS'],ascending=[True, False], inplace=True)

In [100]:
dst_rank_counter = 0

In [101]:
#making a new column for numbering the picks instead of indexing by avg rk
def total_dst_rank(x):
    global dst_rank_counter
    dst_rank_counter +=1
    return dst_rank_counter

In [102]:
#making a new column just counting up from 1 to list as index of picks
combined_ddf['RK'] = combined_ddf['TEAM'].apply(total_dst_rank)
combined_ddf.set_index('RK', inplace=True)

In [103]:
#organizing columns
combined_ddf = combined_ddf.reindex(columns=['TIER',
                                           'TEAM',
                                           'AVG RK',
                                           'FFB RK',
                                           'FPS RK',
                                           'ANDY',
                                           'FAN PTS',
                                           'FINISH',
                                           'DEF TD',
                                           'INT',
                                           'SACK',
                                           'FUMR',
                                           'RET TD',
                                           'SAFE'])

In [104]:
#combined_ddf.to_csv('DST_CombinedRanks.csv')

In [105]:
combined_ddf

,TIER,TEAM,AVG RK,FFB RK,FPS RK,ANDY,FAN PTS,FINISH,DEF TD,INT,SACK,FUMR,RET TD,FUMR,SAFE
RK,,,,,,,,,,,,,,,
1,1,BUF,1.0,1,1,1,150,DST 4,1,19,42,11,0,11,0
2,1,TB,2.0,2,2,3,141,DST 6,2,17,47,12,0,12,0
3,2,DEN,4.5,4,5,5,119,DST 13,1,13,36,6,0,6,0
4,2,LAR,5.5,3,8,2,121,DST 12,0,19,50,6,0,6,0
5,2,NO,6.0,5,7,4,145,DST 5,2,18,46,7,0,7,0
6,2,IND,8.0,12,4,12,138,DST 7,1,19,33,14,0,14,1
7,3,LAC,9.0,7,11,7,78,DST 27,1,11,35,10,0,10,0
8,3,DAL,9.5,10,9,9,185,DST 1,6,26,41,8,0,8,0
9,3,GB,10.5,8,13,11,117,DST 15,2,18,39,8,0,8,0
